In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
from pymongo import MongoClient
import re

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [23]:
import datetime as dt
from bs4 import BeautifulSoup as bs
import requests

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
options.add_argument("lang=ko_KR")

In [4]:
client = MongoClient()
db = client.test
results = db.results_twitter

In [5]:
# 크롬 드라이버의 경로
# headless
add = '/home/jin/다운로드/chromedriver'
driver = webdriver.Chrome(add)
# , chrome_options=options)

In [13]:
keywords = '이대성' # 키워드
words = ['부산', '서울', '광주', '대구', '세종', '제주', '울산', '인천'] # 지역명
startdate = dt.date(year=2019, month=4, day=1)
untildate = dt.date(year=2019, month=4, day=2)
enddate = dt.date(year=2019, month=4, day=10)

In [21]:
url = 'https://twitter.com/search?l=&q='+ keywords +'%20since%3A'+ str(startdate) +'%20until%3A'+ str(end) +'&src=typd'

In [15]:
driver.get(url)

In [17]:
startdate = dt.date(year=2019, month=4, day=1)
untildate = dt.date(year=2019, month=4, day=2)
enddate = dt.date(year=2019, month=4, day=10)

In [29]:
total = []
while not enddate == startdate:
    url='https://twitter.com/search?l=&q=' \
        + keywords +'%20since%3A'+ str(startdate) \
        + '%20until%3A'+ str(end) +'&src=typd'
    
    driver.get(url)

    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    lastH = driver.execute_script('return document.body.scrollHeight')
    
    while True:
        dailyfreq = {'Date':startdate}
        wordfreq = 0
        
        tweets = soup.find_all('p', {'class': 'TweetTextSize'})
        wordfreq = len(tweets)
        
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        
        time.sleep(1)
        
        newH = driver.execute_script('return document.body.scrollHeight')
        print(newH)
        if newH != lastHeight:
            html = browser.page_source
            soup = bs(html, 'html.parser')
            tweets = soup.find_all('p', {'class': 'TweetTextSize'})
            wordfreq = len(tweets)
        else:
            dailyfreq['Freq'] = wordfreq
            wordfreq = 0
            total.append(dailyfreq)
            startdate = untildate
            untildate += dt.timedelta(days=1)
            dailyfreq = {}
            
            break
        lastHeight = newHeight
    
    

2969


In [20]:
str(startdate)

'2019-04-01'

In [30]:
tweets = soup.find_all('p', {'class': 'TweetTextSize'})

In [38]:
for t in tweets:
    print(t)

<p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="ko">[투데이★] 꿈의 무대 다시 찾는 <strong>이대성</strong> “나에게 약했던 팟츠, 다시 막겠다” <a class="twitter-timeline-link" data-expanded-url="http://m.jumpball.co.kr/view.php?no=73260" dir="ltr" href="https://t.co/bcBoDrPgxa" rel="nofollow noopener" target="_blank" title="http://m.jumpball.co.kr/view.php?no=73260"><span class="tco-ellipsis"></span><span class="invisible">http://</span><span class="js-display-url">m.jumpball.co.kr/view.php?no=73</span><span class="invisible">260</span><span class="tco-ellipsis"><span class="invisible"> </span>…</span></a></p>
<p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="ko">8분에 <strong>이대성</strong> 3점 암만봐도 3점인데 무슨 비디오판독하고도 2점이라네 홈콜 ㅅㅂ 정의구현 잘했네 <a class="twitter-timeline-link" data-expanded-url="http://naver.me/G6KTPLSd" dir="ltr" href="https://t.co/RTHy3U8UBj" rel="nofollow noopener" target="_blank" title="http://naver.me/G6KTPLSd"><span class="tco-ellipsis"></sp